In [7]:
import os
print(os.getcwd())        # 현재 작업 디렉토리 출력
print(os.listdir('.'))    # 현재 폴더에 있는 파일 목록 출력

D:\Workspace\Paper\합성 데이터
['.ipynb_checkpoints', 'merge_card gyeonggi data_202509.csv', 'merge_commercial area_20251030.csv', 'merge_seoul_purpose_admdong1_in_202509.csv', 'Untitled.ipynb']


In [ ]:
# 1. 라이브러리
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

# 2. 실제 데이터 로드 (파일명만 바꾸세요)
stores = pd.read_csv("merge_commercial area_20251030.csv", low_memory=False)
card   = pd.read_csv("merge_card gyeonggi data_202509.csv", chunksize=1_000, low_memory=False)
mob    = pd.read_csv("merge_seoul_purpose_admdong1_in_202509.csv", low_memory=False)

# 3. 대상 상점 필터링 (소매: 식료품/음료/종합 + 음식 전체)
store_retail = stores[
    (stores['상권업종대분류명'] == '소매') &
    (stores['상권업종중분류명'].isin(['식료품 소매', '음료 소매', '종합 소매']))
]
store_food = stores[stores['상권업종대분류명'] == '음식']
target_stores = pd.concat([store_retail, store_food], ignore_index=True)[['상호명','위도','경도','상권업종소분류명']]
print(f"대상 상점: {len(target_stores):,}개")

# 간단 카테고리 매핑
def simple_cat(x):
    if any(k in x for k in ['편의점','마트','슈퍼']) : return '편의점'
    if any(k in x for k in ['피자','치킨']) : return '피자'
    if any(k in x for k in ['제과','베이커리','빵']) : return '빵'
    if '샐러드' in x : return '샐러드'
    if any(k in x for k in ['도시락','김밥']) : return '도시락'
    return '기타음식'
target_stores['cat'] = target_stores['상권업종소분류명'].apply(simple_cat)

# 4. 카드 소비 데이터에서 실제 구매 패턴 추출 (한 번만 실행)
print("카드 데이터에서 구매 패턴 추출 중...")
card_dist = pd.DataFrame()
for chunk in card:
    tmp = chunk[chunk['card_tpbuz_nm_1'].isin(['소매/유통', '음식'])]
    card_dist = pd.concat([card_dist, tmp.sample(frac=0.02, random_state=42)])
card_dist = card_dist.head(500_000)  # 메모리 고려

# 시간대별 구매 비율
hour_weight = card_dist['hour'].value_counts(normalize=True).sort_index()
# 금액 분포 (원가 기준)
price_dist = card_dist['amt'].astype(float)

# 5. 생활이동 도착지 → 사용자 위치로 사용
dest_points = mob[['위도','경도']].dropna().sample(1_000, replace=True, random_state=42).values

# 6. 100만 건 합성 데이터 생성 (가장 짧고 빠름)
n = 10_000
data = []

print("100만 건 합성 데이터 생성 시작...")
for i in tqdm(range(n)):
    # 사용자 현재 위치 = 생활이동 도착지 근처 300m 반경
    lat, lon = dest_points[i]
    user_lat = lat + np.random.normal(0, 0.0015)  # 약 150~300m 오차
    user_lon = lon + np.random.normal(0, 0.0015)
    
    # 상점 랜덤 선택
    store = target_stores.sample(1).iloc[0]
    dist = ((user_lat - store['위도'])**2 + (user_lon - store['경도'])**2)**0.5 * 111  # km
    
    # 실제 카드 데이터 기반 변수
    hour = int(np.random.choice(hour_weight.index, p=hour_weight.values))
    discount = np.clip(np.random.beta(2, 3)*70, 20, 70)      # 실제 할인율 분포 (평균 45%)
    original_price = max(3000, np.random.choice(price_dist, 1)[0])
    hours_left = np.random.triangular(1, 8, 24)
    
    # 구매 확률 (실제 카드 데이터 반영 + 거리/할인/시간대)
    logodds = -0.8*dist + 0.07*discount + 0.03*hours_left + (2 if hour>=18 else 0)
    prob = 1/(1+np.exp(-logodds))
    purchased = 1 if np.random.rand() < prob else 0
    
    data.append({
        'user_lat': user_lat, 'user_lon': user_lon,
        'store_lat': store['위도'], 'store_lon': store['경도'],
        'distance_km': round(dist, 3),
        'discount_rate': round(discount, 1),
        'hours_to_expiry': round(hours_left, 1),
        'original_price': int(original_price),
        'discounted_price': int(original_price * (1 - discount/100)),
        'category': store['cat'],
        'hour': hour,
        'time_slot': '저녁' if 18<=hour<=21 else '점심' if 11<=hour<=14 else '아침' if 6<=hour<=10 else '기타',
        'purchased': purchased
    })

# 7. 저장
df = pd.DataFrame(data)
df = pd.get_dummies(df, columns=['category', 'time_slot'])  # 모델링용
df.to_csv("real_synthetic_1M_dataset.csv", index=False)
print(f"\n완료! 1,000,000건 저장 → real_synthetic_1M_dataset.csv")
print(f"실제 구매율: {df['purchased'].mean():.1%}")
print(f"저녁 시간대 구매율: {df[df['hour'].between(18,21)]['purchased'].mean():.1%}")

대상 상점: 393,359개
카드 데이터에서 구매 패턴 추출 중...


In [14]:
df=pd.read_csv("merge_commercial area_20251030.csv")
print(df.head())

                 상가업소번호        상호명  지점명 상권업종대분류코드 상권업종대분류명 상권업종중분류코드  \
0  MA010120220813245971    씨유평택고평점  NaN        G2       소매      G204   
1  MA010120220800140341       더빈티지  NaN        G2       소매      G222   
2  MA010120220806717520        반트랑  NaN        I2       음식      I205   
3  MA010120220806874384     에스인테리어  NaN        M1    과학·기술      M112   
4  MA010120220802338302  청솔랜드공인중개사  NaN        L1      부동산      L102   

   상권업종중분류명 상권업종소분류코드    상권업종소분류명 표준산업분류코드  ...                     건물관리번호  \
0     종합 소매    G20405         편의점   G47122  ...  4122011400102310001000001   
1  중고 상품 소매    G22201   중고 상품 소매업   G47869  ...  4119910500103110007009478   
2     동남아시아    I20501     베트남식 전문   I56129  ...  4119510800110710006003364   
3    전문 디자인    M11201   인테리어 디자인업   F42412  ...  4115010900102260010019256   
4   부동산 서비스    L10203  부동산 중개/대리업   L68221  ...  4127310100107730001019737   

    건물명                    도로명주소     구우편번호  신우편번호  동정보  층정보  호정보          경도  \
0   NaN          경